In [3]:
# !git clone https://github.com/google-research/pegasus
%cd /content/pegasus
!export PYTHONPATH=.

/content/pegasus


In [4]:
!sudo apt-get install gcc python-dev python-setuptools libffi-dev
!sudo apt-get install python-pip
!sudo pip install gsutil

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libffi-dev is already the newest version (3.2.1-8).
python-dev is already the newest version (2.7.15~rc1-1).
python-setuptools is already the newest version (39.0.1-2).
gcc is already the newest version (4:7.4.0-1ubuntu2.3).
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-pip is already the newest version (9.0.1-2.3~ubuntu1.18.04.2).
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.


In [5]:
pip install -r requirements.txt

In [7]:
!mkdir ckpt
%cd ckpt
!mkdir pegasus_ckpt
%cd /content/pegasus

mkdir: cannot create directory ‘ckpt’: File exists
/content/pegasus/ckpt
/content/pegasus


In [8]:
!gsutil cp -r gs://pegasus_ckpt/checkpoint ckpt/pegasus_ckpt
!gsutil cp -r gs://pegasus_ckpt/c4.unigram.newline.10pct.96000.model ckpt/pegasus_ckpt
!gsutil cp -r gs://pegasus_ckpt/c4.unigram.newline.10pct.96000.vocab ckpt/pegasus_ckpt
!gsutil cp -r gs://pegasus_ckpt/model.ckpt-1500000.meta ckpt/pegasus_ckpt
!gsutil cp -r gs://pegasus_ckpt/model.ckpt-1500000.index ckpt/pegasus_ckpt
!gsutil cp -r gs://pegasus_ckpt/model.ckpt-1500000.data-00000-of-00001 ckpt/pegasus_ckpt

Copying gs://pegasus_ckpt/checkpoint...
/ [1 files][   93.0 B/   93.0 B]                                                
Operation completed over 1 objects/93.0 B.                                       
Copying gs://pegasus_ckpt/c4.unigram.newline.10pct.96000.model...
/ [1 files][  1.8 MiB/  1.8 MiB]                                                
Operation completed over 1 objects/1.8 MiB.                                      
Copying gs://pegasus_ckpt/c4.unigram.newline.10pct.96000.vocab...
/ [1 files][  1.7 MiB/  1.7 MiB]                                                
Operation completed over 1 objects/1.7 MiB.                                      
Copying gs://pegasus_ckpt/model.ckpt-1500000.meta...
/ [1 files][220.4 MiB/220.4 MiB]                                                
Operation completed over 1 objects/220.4 MiB.                                    
Copying gs://pegasus_ckpt/model.ckpt-1500000.index...
/ [1 files][ 16.7 KiB/ 16.7 KiB]                                     

In [9]:
!gsutil cp -r gs://pegasus_ckpt/xsum  ckpt/pegasus_ckpt

Copying gs://pegasus_ckpt/xsum/model.ckpt-30000.data-00000-of-00001...
Copying gs://pegasus_ckpt/xsum/model.ckpt-30000.index...
Copying gs://pegasus_ckpt/xsum/model.ckpt-30000.meta...
\ [3 files][  2.1 GiB/  2.1 GiB]   35.1 MiB/s                                   
Operation completed over 3 objects/2.1 GiB.                                      


In [10]:
%cd /content

/content


In [11]:
!pip3 install -e pegasus

Obtaining file:///content/pegasus
  Found existing installation: pegasus 0.0.1
    Can't uninstall 'pegasus'. No files were found to uninstall.
  Running setup.py develop for pegasus


In [17]:
%cd /content/pegasus/

/content/pegasus


In [23]:
inp = """
      Federal Reserve Chairman Jerome Powell said the U.S. economy is improving, but it will take a long way to fully recover from the effects of the new coronavirus pandemic.
　　The text of his testimony to the House Financial Services Committee, scheduled for March 22, has been released. According to it, Chairman Powell states that "a number of economic indicators are showing marked improvement".
　　He goes on to acknowledge that "both employment and general economic activity remain well below their pre-pandemic levels, and the outlook remains extremely uncertain.
　　Chairman Powell and Treasury Secretary Mnuchin will provide quarterly congressional testimony as required by the Coronavirus Assistance, Relief, and Economic Security (CARES) Act, which was passed by Congress in March.
　　According to the text of his testimony, he will reiterate his view that further policy responses, both fiscal and monetary, will be needed to avert long-term damage to the economy from a pandemic, stating that "the future depends on the containment of the coronavirus and policy actions at all levels of government.
　　Powell is also anticipating questions surrounding the lack of widespread use of the Fed's small business assistance program, the Main Street Lending Program (MSLP).
　　He said he has responded to feedback by making adjustments to the program, but added: "Main Street lending may not be the right solution for some businesses. This is partly because the CARES Act clearly states that these loans cannot be written off," he noted.
"""

In [19]:
# save_path = "tfrecord:/content/pegasus/pegasus/data/testdata/test_pattern_1.tfrecord"
# @registry.register("reddit_tifu_long_transformer")
# def reddit_tifu_long_transformer(param_overrides):
#  return transformer_params(
#  {
#  "train_pattern": save_path, 
#  "dev_pattern": save_path, 
#  "test_pattern": save_path, 
#  "max_input_len": 1024,
#  "max_output_len": 128,
#  "train_steps": 8000,
#  "learning_rate": 0.0001,
#  "batch_size": 8,
#  }, param_overrides)

NameError: ignored

In [24]:
import pandas as pd
import tensorflow as tf

input_dict = dict(
    inputs=[inp],
    targets=[""]
    )

save_path = "pegasus/data/testdata/test_pattern_1.tfrecord"
data = pd.DataFrame(input_dict)
with tf.io.TFRecordWriter(save_path) as writer:
    for row in data.values:
        inputs, targets = row[:-1], row[-1]
        example = tf.train.Example(
            features=tf.train.Features(
                feature={
                    "inputs": tf.train.Feature(bytes_list=tf.train.BytesList(value=[inputs[0].encode('utf-8')])),
                    "targets": tf.train.Feature(bytes_list=tf.train.BytesList(value=[targets.encode('utf-8')])),
                }
            )
        )
        writer.write(example.SerializeToString())

In [25]:
!python3 pegasus/bin/evaluate.py \
--params=xsum_test \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=1,beam_size=2,beam_alpha=0.6 \
--model_dir=ckpt/pegasus_ckpt/xsum/model.ckpt-30000



Instructions for updating:
Use standard file APIs to check for files with this prefix.
W0922 03:46:19.463682 140456535021440 deprecation.py:323] From pegasus/bin/evaluate.py:85: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.

W0922 03:46:19.466633 140456535021440 module_wrapper.py:139] From /content/pegasus/pegasus/ops/public_parsing_ops.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0922 03:46:19.627885 140456535021440 module_wrapper.py:139] From /content/pegasus/pegasus/params/estimator_utils.py:49: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0922 03:46:19.628487 140456535021440 estimator.py:1994] Estimator's model_fn (<function _estimator_model_fn.<locals>.model_fn at 0x7fbe150e0730>) includes params argument, but params are not passe